In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
from ldmm import LDMM
from same import SAME
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
from tqdm import tqdm
from statsmodels.tsa.arima_model import ARIMA
from IPython.display import clear_output
import itertools
import seaborn as sns
sns.set()

### Preprocessing

In [2]:
df = pd.read_excel('real_data.xls', names=['Время',\
'Реальные располагаемые денежные доходы в % к предущему месяцу',\
'Реальные располагаемые денежные доходы в % к соответствующему периоду предыдущего года',\
'Реальные располагаемые денежные доходы база январь 1999 =100',\
'Реальная начисленная заработная плата одного работника в % к предыдущему месяцу',\
'Реальная начисленная заработная плата одного работника база январь 1999 =100'])
df = df.dropna()

In [3]:
data = np.array(df)[:, 1:]
data[55:60, 0] = [99.8, 101.0, 106.1, 100.1, 130.4]
data[55:60, 3] = [98.3, 100.9, 104.6, 101.1, 121.3]
data[61, 3] = 102.3
timestamps = np.arange(data.shape[0])

In [4]:
data = data[:, [0,1,2,3,4]]
print(data.shape)

(238, 5)


In [5]:
mask = np.tile(np.append(np.ones(1), np.append(np.zeros(10), np.ones(1))), 20)[:-2]
data = data[mask==0]
timestamps = timestamps[mask==0]

### Predictor

In [6]:
# parameters


In [7]:
# Convert a time series to large-dimensional vectors
#
# ts -- 2-dimensional array, pepresenting a multivariate time series
# bandwidth -- size of the bandwidth
#
# returns: an array of large-dimensional vectors

def ts_to_vec(ts, bandwidth=8):
    
    vec = ts[:, :]
    for i in range(bandwidth-1):
        vec = np.append(vec[:-1, :ts.shape[1]], vec[1:, :], axis=1)
        
    return vec

In [8]:
def generalized_features(Y_train, bandwidth=12):
    
    # construct the generalized states
    Y_gen = ts_to_vec(Y_train, bandwidth=bandwidth)
    # split the generalized data onto new train and test features
    X_train = Y_gen[:-1, :]
    X_test = Y_gen[-1, :].reshape(-1)
    
    return X_train, X_test

In [9]:
# Localizing kernel for computation of weights
def loc_kernel(x):
    # Gaussian kernel
    #return np.exp(-0.5*x**2)
    # Epanechnikov kernel
    return np.maximum(1 - x**2, 0)

In [10]:
def shifted_weighted_kNN(timestamps_train, timestamps_test, X_train, Y_train, X_test, n_neighbors, kernel=loc_kernel, lambd=0.05):
    
    # increments
    n_train = Y_train.shape[0]
    A = np.diag(np.ones(n_train), 0) - np.diag(np.ones(n_train-1), -1)
    A[0,0] = 0
    increments = A.dot(Y_train)
    Y_shifted = increments + Y_train[-1]
    #Y_shifted = Y_train
    
    # predictions
    Y_pred = np.empty(Y_train.shape[1])
    # compute distances
    dist = np.linalg.norm(X_train - X_test, axis=1)
    # sort distances
    sorted_dist = np.sort(dist)
    
    for i in range(Y_train.shape[1]):
        # compute weights
        h = sorted_dist[n_neighbors[i]]
        weights = kernel(dist / h)
        # reweight by the time
        weights = weights * np.exp(lambd * (timestamps_train - timestamps_test)) 

        # use weighted kNN for the prediction
        Y_pred[i] = np.sum(Y_shifted[:, i] * weights) / np.sum(weights)
    #Y_pred = (Y_shifted.transpose().dot(weights)).transpose() / np.sum(weights)
    
    return Y_pred

In [11]:
def predict_LDMM(timestamps_train, Y_train, timestamp_test, params, n_neighbors):
    bandwidth, lambd, mu, h, n_iteration = params
    #print(Y_train.shape)
    # Construct the generalized features
    generalized_X_train, generalized_X_test = generalized_features(Y_train, bandwidth=bandwidth)
    
    # Construct a manifold
    generalized_X = np.append(generalized_X_train, generalized_X_test.reshape(1, -1), axis=0).astype(float)
    # Normalize the numerical features
    generalized_X = normalize(generalized_X)
    # Find a manifold
    Z = LDMM(generalized_X, lambd=lambd, mu=mu, h=h, n_iterations=n_iteration, b=0)
    #n_iterations = 10
    #neighbors_list = np.array([50 * 0.93**i for i in range(n_iterations)]).astype(int)
    #Z = SAME(generalized_X, neighbors_list)
    
    #print(np.linalg.norm(Z - generalized_X))
    #print(np.linalg.norm(generalized_X))
    #print(np.linalg.norm(Z))
    
    # Define modified train and test features
    Z_train = Z[:-1, :]
    Z_test = Z[-1, :]
    
    #print(Z_train.shape, timestamps_train[bandwidth:].shape, Y_train[bandwidth:].shape)
    predictions = shifted_weighted_kNN(timestamps_train[bandwidth:], timestamp_test,\
                                       Z_train, Y_train[bandwidth:], Z_test, n_neighbors=n_neighbors)
        
    return predictions

In [12]:
def predict_SAME(timestamps_train, Y_train, timestamp_test, params):
    bandwidth, tau, n_iterations, neighbors1, neighbors2, neighbors3, neighbors4, neighbors5= params
    n_neighbors = (neighbors1, neighbors2, neighbors3, neighbors4, neighbors5)
    #print(Y_train.shape)
    # Construct the generalized features
    generalized_X_train, generalized_X_test = generalized_features(Y_train, bandwidth=bandwidth)
    
    # Construct a manifold
    generalized_X = np.append(generalized_X_train, generalized_X_test.reshape(1, -1), axis=0).astype(float)
    # Normalize the numerical features
    generalized_X = normalize(generalized_X)
    Z = generalized_X

    # Find a manifold
    #Z = LDMM(generalized_X, lambd=10, mu=0.2 * generalized_X.shape[0] * 0.1, h=0.1, n_iterations=10, b=0)
#     n_iterations = 10
    neighbors_list = np.array([50 * 0.93**i for i in range(n_iterations)]).astype(int)
    Z = SAME(generalized_X, neighbors_list, tau)
    
    #print(np.linalg.norm(Z - generalized_X))
    #print(np.linalg.norm(generalized_X))
    #print(np.linalg.norm(Z))
    
    # Define modified train and test features
    Z_train = Z[:-1, :]
    Z_test = Z[-1, :]
    
    #print(Z_train.shape, timestamps_train[bandwidth:].shape, Y_train[bandwidth:].shape)
    predictions = shifted_weighted_kNN(timestamps_train[bandwidth:], timestamp_test,\
                                       Z_train, Y_train[bandwidth:], Z_test, n_neighbors=n_neighbors)
        
    return predictions

In [13]:
def predict_knn(timestamps_train, Y_train, timestamp_test, n_neighbors, bandwidth=12):
    
    #print(Y_train.shape)
    # Construct the generalized features
    generalized_X_train, generalized_X_test = generalized_features(Y_train, bandwidth=bandwidth)
    
    # Construct a manifold
    generalized_X = np.append(generalized_X_train, generalized_X_test.reshape(1, -1), axis=0).astype(float)
    # Normalize the numerical features
    Z = normalize(generalized_X)
    #print(np.linalg.norm(Z - generalized_X))
    #print(np.linalg.norm(generalized_X))
    #print(np.linalg.norm(Z))
    
    # Define modified train and test features
    Z_train = Z[:-1, :]
    Z_test = Z[-1, :]
    
    #print(Z_train.shape, timestamps_train[bandwidth:].shape, Y_train[bandwidth:].shape)
    predictions = shifted_weighted_kNN(timestamps_train[bandwidth:], timestamp_test,\
                                       Z_train, Y_train[bandwidth:], Z_test, n_neighbors=n_neighbors)
        
    return predictions

### Test

In [16]:
titles = ['Real disposable cash income in % \n to the previous month', \
          'Real disposable cash income in % \n to the corresponding period of the previous year', \
          'Real disposable cash income \n base January 1999 = 100', \
          'Real accrued wages \n of one employee in% of the previous month', \
          'Real accrued wages \n one worker base January 1999 = 100']

In [20]:
# Number of test months
n_test = 40
params=(11, 7.0, 1500.0, 0.001, 7)
params_knn = (11, 100.0, 1000.0, 1000.0, 15)
lookfront = 1

In [42]:
# losses_knn  = []   
# losses_ldmm  = []  
# (3, 1.0, 21, )
nn = [5, 60, 10, 7, 7]
nn_same = [3, 9, 21, 21, 15]
nn_knn = [30, 5, 30, 30, 30]
losses = []
for lookfront in [1,2,3,4]:
# for j, params in enumerate(itertools.product(bandwidths, lambdas, mus, hs, n_iterations)):
    predictions_knn  = np.empty((n_test, lookfront, 5))
    predictions_ldmm  = np.empty((n_test, lookfront, 5))
    predictions_same  = np.empty((n_test, lookfront, 5))
    predictions_arima  = np.empty((n_test, lookfront, 5))

    outcomes = np.empty((n_test, lookfront, 5))
    for i in range(n_test):

        Y_train_knn = data[:-n_test+i+1-lookfront, :]
        Y_train_ldmm = data[:-n_test+i+1-lookfront, :]
        Y_train_same = data[:-n_test+i+1-lookfront, :]
        Y_train_arima = data[:-n_test+i+1-lookfront, :]

        for k in range(lookfront):
            timestamps_train = timestamps[:-n_test+i+1-lookfront+k]
            timestamp_test = timestamps[-n_test+i+1-lookfront+k]
            Y_test = data[-n_test+i+1-lookfront+k, :]

            predictions_knn[i, k, :] = predict_knn(timestamps_train, Y_train_ldmm, timestamp_test, params_knn, n_neighbors=nn_knn)[:]#predict_knn(timestamps_train, Y_train_knn, timestamp_test, n_neighbors=nn)[:]
            predictions_ldmm[i, k, :] = predict_LDMM(timestamps_train, Y_train_ldmm, timestamp_test, params, n_neighbors=nn)[:]
            predictions_same[i, k, :] = predict_SAME(timestamps_train, Y_train_same, timestamp_test, (3, 1.0, 21,*nn_same))[:]
            for t in range(5):
#                 if t == 0:
#                     predictions_arima[i, k, t] = ARIMA(Y_train_arima[:, t], order=(6,2,0)).fit(disp=0, trend='nc').forecast(steps=1)[0]
                predictions_arima[i, k, t] = ARIMA(Y_train_arima[:, t], order=(6,1,0)).fit(disp=0, trend='nc').forecast(steps=1)[0]


            outcomes[i, k, :] = Y_test[:]
            Y_train_knn = np.append(Y_train_knn, predictions_knn[i, k, :].reshape(1,-1), axis=0)
            Y_train_ldmm = np.append(Y_train_ldmm, predictions_ldmm[i, k, :].reshape(1,-1), axis=0)
            Y_train_same = np.append(Y_train_same, predictions_same[i, k, :].reshape(1,-1), axis=0)
            Y_train_arima = np.append(Y_train_arima, predictions_arima[i, k, :].reshape(1,-1), axis=0)


    new_loss_knn = np.mean((predictions_knn[:,-1]-outcomes[:,-1])**2/ outcomes[:,-1]**2, axis=0)
    new_loss_ldmm = np.mean((predictions_ldmm[:,-1]-outcomes[:,-1])**2 / outcomes[:,-1]**2, axis=0)
    new_loss_same = np.mean((predictions_same[:,-1]-outcomes[:,-1])**2 / outcomes[:,-1]**2, axis=0)
    new_loss_arima = np.mean((predictions_arima[:,-1]-outcomes[:,-1])**2 / outcomes[:,-1]**2, axis=0)
    #maybe it calculates incorrect
    std_knn = np.sqrt(np.mean(((predictions_knn[:,-1]-outcomes[:,-1])**2/ outcomes[:,-1]**2 - new_loss_knn) ** 2, axis=0))
    std_ldmm = np.sqrt(np.mean(((predictions_ldmm[:,-1]-outcomes[:,-1])**2/ outcomes[:,-1]**2 - new_loss_ldmm) ** 2, axis=0))
    std_same = np.sqrt(np.mean(((predictions_same[:,-1]-outcomes[:,-1])**2/ outcomes[:,-1]**2 - new_loss_same) ** 2, axis=0))
    std_arima = np.sqrt(np.mean(((predictions_arima[:,-1]-outcomes[:,-1])**2/ outcomes[:,-1]**2 - new_loss_arima) ** 2, axis=0))

# parameters.append(params)
# losses.append(np.mean(new_loss_ldmm))
# clear_output(wait=True)
# print(j, np.mean(new_loss_ldmm), params, new_loss_ldmm)
# minset = np.argmin(losses)
# print('minimum:', minset, losses[minset], parameters[minset])
    print(lookfront)
    print('knn', new_loss_knn[1:])
    print('ldmm', new_loss_ldmm[1:])
    print('same', new_loss_same[1:])
    print('arima', new_loss_arima[1:])
    print('knn', std_knn[1:])
    print('ldmm', std_ldmm[1:])
    print('same', std_same[1:])
    print('arima', std_arima[1:])
    print()
#     for i in range(5):
#         plt.plot(timestamps[-n_test:], predictions_ldmm[:, -1, i], 'b--', label='LDMM')
#         plt.plot(timestamps[-n_test:], predictions_knn[:, -1, i], 'g--', label='kNN')
#         plt.plot(timestamps[-n_test:], predictions_same[:, -1, i], 'y--', label='SAME')
#         plt.plot(timestamps[-n_test:], predictions_arima[:, -1, i], 'r--', label='ARIMA')
#         plt.plot(timestamps[-n_test:], data[-n_test:, i], 'k-', label='Test data')
#         plt.legend(loc=3,fontsize='xx-small')
#         plt.title(titles[i])
#         plt.xlabel('Month')
#         plt.ylabel('Value')
#         plt.savefig("LDMM_ranepa_multivariate_lookfront="+str(lookfront)+str(i)+".png")
#         plt.show()
# losses_ldmm.append(new_loss_ldmm)
# losses_knn.append(new_loss_knn)

# #5 1000 10000 10000 плохо
# 18521 0.004342346107326652 (17, 10000.0, 10000.0, 10000.0, 30) [0.01537566 0.0011506  0.00255974 0.00158956 0.00103616]
# minimum: 9324 0.003404438586616477 (11, 10.0, 1000.0, 0.001, 5)

1
knn [0.00165557 0.00396715 0.00239861 0.00177703]
ldmm [0.00155454 0.00241133 0.00152581 0.00142244]
same [0.00116738 0.0018553  0.00151837 0.0016329 ]
arima [0.00837624 0.00213298 0.00145582 0.00158337]
knn [0.00233372 0.00534879 0.00410975 0.00357277]
ldmm [0.00159845 0.00330324 0.00266793 0.00196078]
same [0.00196213 0.00455778 0.0031503  0.00306884]
arima [0.00118552 0.00305858 0.00251821 0.00269182]

2
knn [0.00176472 0.00434126 0.00303796 0.00427745]
ldmm [0.0014592 0.0028675 0.00198019 0.0042891]
same [0.001458 0.00245318 0.00209719 0.00394271]
arima [0.00176388 0.00249325 0.00194276 0.00375188]
knn [0.00170855 0.00590817 0.00411123 0.00499822]
ldmm [0.00158091 0.00235454 0.00353042 0.00354393]
same [0.00205441 0.00569466 0.00393462 0.00502488]
arima [0.00117109 0.00391763 0.00250408 0.00413675]

3
knn [0.0030487  0.00709648 0.0042467  0.00584049]
ldmm [0.00281025 0.00402681 0.00293258 0.00486037]
same [0.00189623 0.00334985 0.00278699 0.00510701]
arima [0.00183599 0.0031047  